## Importe de paquetes

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, StringType, IntegerType, DateType
from pyspark.sql.functions import udf, col, length, isnan, when, count
import pyspark.sql.functions as f
import os 
from collections import Counter
from datetime import datetime
from pyspark.sql import types as t
#from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## Configuración

### Configuración de la sesion Spark

In [2]:
path_jar_driver = '../mysql-connector-java-8.0.28.jar'

In [3]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

22/09/07 18:42:08 WARN Utils: Your hostname, Ricardos-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.3 instead (on interface en0)
22/09/07 18:42:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/07 18:42:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/riarangogi/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


### Configuración de la conexión a la BD

In [4]:
# Si quiere practicar la conexion con el servidor de base de datos:
db_connection_string = 'jdbc:mysql://157.253.236.116:8080/ProyectoTransaccional'
# El usuario es su estudiante _i asignado y su contraseña la encontrará en el archivo excel de Coursera 
db_user = 'Estudiante_4_202214'
db_psswd = 'NT8ZK03Q4H'

PATH='./'

In [5]:
sql_aeropuertos = 'ProyectoTransaccional.aeropuertosCopia'

In [6]:
sql_vuelos = 'ProyectoTransaccional.vuelosCopia2'

In [7]:
sql_proyecciones = 'ProyectoTransaccional.proyeccionesCopia'

In [8]:
sql_divipola = 'ProyectoTransaccional.divipolaCopia'

In [9]:
sql_pib = 'ProyectoTransaccional.PIBCopia'

## Cargue de datos

In [10]:
def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

### Tabla aeropuertosCopia

In [11]:
aeropuertos = obtener_dataframe_de_bd(db_connection_string, sql_aeropuertos, db_user, db_psswd)

In [12]:
aeropuertos.show(5)

+-----+----+--------------------+--------------------+------------+----------+-------+--------+-----------------+--------------------+--------------+-----------+------+-----------+---------+----------+------------------+--------------+-----+--------+--------------------+----------------+-------------+
|sigla|iata|              nombre|           municipio|departamento| categoria|latitud|longitud|      propietario|          explotador|longitud_pista|ancho_pista|  pbmo|orientacion|elevacion|resolucion|fecha_construccion|fecha_vigencia|clase|    tipo|numero_vuelos_origen|gcd_departamento|gcd_municipio|
+-----+----+--------------------+--------------------+------------+----------+-------+--------+-----------------+--------------------+--------------+-----------+------+-----------+---------+----------+------------------+--------------+-----+--------+--------------------+----------------+-------------+
|  9cg|    |san jose del ariporo|      Paz de Ariporo|    casanare|AerÃ³dromo|    500|-70.0

### Tabla vuelosCopia

In [13]:
vuelos = obtener_dataframe_de_bd(db_connection_string, sql_vuelos, db_user, db_psswd)

In [14]:
vuelos.show(5)

+----+---+------+-------+-----------+----------+-------+--------+------+------+--------------+---------+-----------+
| ano|mes|origen|destino|tipo_equipo|tipo_vuelo|trafico| empresa|vuelos|sillas|carga_ofrecida|pasajeros|carga_bordo|
+----+---+------+-------+-----------+----------+-------+--------+------+------+--------------+---------+-----------+
|2004|  1|   bog|    mia|       B752|         R|      I|AMERICAN|     0|     0|           0.0|        0|        0.0|
|2004|  7|   bog|    mia|       B752|         R|      I|AMERICAN|    62| 11656|      327236.0|     7864|   118443.0|
|2004|  9|   bog|    mia|       B752|         R|      I|AMERICAN|    32|  6016|      168896.0|     4283|    73896.0|
|2004|  3|   bog|    mia|       B763|         R|      I|AMERICAN|    31|  6572|      351540.0|     3473|   203724.0|
|2004|  7|   bog|    mia|       B763|         R|      I|AMERICAN|    31|  6572|      351540.0|     4994|   219965.0|
+----+---+------+-------+-----------+----------+-------+--------

### Tabla proyeccionesCopia

In [15]:
proyecciones = obtener_dataframe_de_bd(db_connection_string, sql_proyecciones, db_user, db_psswd)

In [16]:
proyecciones.show(5)

+----+---+---------+-------------+-------------+
|Anio| DP|    DPNOM|Total Hombres|Total Mujeres|
+----+---+---------+-------------+-------------+
|2005| 25|antioquia|      2630787|      2830056|
|2006| 25|antioquia|      2663957|      2864164|
|2007| 25|antioquia|      2697202|      2897978|
|2008| 25|antioquia|      2730505|      2931594|
|2009| 25|antioquia|      2764235|      2966142|
+----+---+---------+-------------+-------------+
only showing top 5 rows



### Tabla divipolaCopia

In [17]:
divipola = obtener_dataframe_de_bd(db_connection_string, sql_divipola, db_user, db_psswd)

In [18]:
divipola.show(5)

+-------------------+----------------+---------------------+-------------------+----------------+---------------------+-------------------+--------------+--------------+---------------+----------------------------------------+-------------------------+
|Código Departamento|Código Municipio|Código Centro Poblado|Nombre Departamento|Nombre Municipio|Nombre Centro Poblado|Tipo Centro Poblado|      Longitud|       Latitud|Nombre Distrito|Municipio/Áreas No Municipalizadas (ANM)|Nombre Área Metropolitana|
+-------------------+----------------+---------------------+-------------------+----------------+---------------------+-------------------+--------------+--------------+---------------+----------------------------------------+-------------------------+
|                 41|           91001|             91001033|           amazonas|           NEIVA| COMUNIDAD INDÍGEN...|     CENTRO POBLADO|-70.0285785394|-4.11198257304|               |                               MUNICIPIO|               

### Tabla pibCopia

In [19]:
pib = obtener_dataframe_de_bd(db_connection_string, sql_pib, db_user, db_psswd)

In [20]:
pib.show(5)

+-------------------------------+-------------+-------+--------+--------------------+--------------------+--------+--------+--------------------+--------+
|CÃ³digo Departamento (DIVIPOLA)|DEPARTAMENTOS|   2006|    2007|                2008|                2010|    2011|    2012|                2013|    2014|
+-------------------------------+-------------+-------+--------+--------------------+--------------------+--------+--------+--------------------+--------+
|                             41|         uila|6895658| 7441987|   8686536.103258422|    9548624.85406653|11113741|11400169|1.1967750469189592E7|12755711|
|                             18|     CaquetÃ¡|3575615| 4060678|   4519574.129383702|   5048396.606737779| 5516404| 6476778|    7161524.44383983| 7990249|
|                              8|   AtlÃ¡ntico|7194985| 8299825|   8858405.054387597|   9481200.322350295|10288314|11421539|1.2434114309437651E7|13681473|
|                             63|     QuindiÃ³|6350376| 6908084|  7449

# Perfilamiento de los datos

El grupo de consultores de analitica descriptiva de Wide World Importers asegura que las tablas Grupo de compras, Ciudades, Colores, Paises, Clientes, Categorias de clientes, Paquetes, Estados y provincias, Productos y Personas ya fueron perfiladas y corregidos; sin embargo le recomiendan revisar el diccionario suministrado dado que todas las tablas serán utilizadas en futuros requerimientos.
Si tienen alguna duda relacionada con estas tablas deben incluirla en las preguntas o aclaraciones que le entreguen a la organización como parte del resultado de esta fase de entendimiento.

En este punto de la consultorìa, nuestra responsabilidad es analizar la tabla movimientosCopia, que registra el movimiento en inventario de un producto. La empresa no compartió información de las tablas de clientes ni de proveedores para esta actividad. Con respecto a esta fuente nos comentan lo siguiente:

- La cantidad máxima de productos movidos es 50 millones por transacción
- Existen 236.668 movimientos de producto realizados desde 2013
- Los movimientos están relacionados con 228.265 clientes
- El formato de fechas manejado es YYYY-MM-DD HH:MM:SS
- Actualmente tenemos 13 proveedores


## ¿Cuantos datos nos compartieron para el proyecto?

### Tabla aeropuertosCopia

In [21]:
print(f"registros = {aeropuertos.count()}", f"columnas = {len(aeropuertos.columns)}")

registros = 547 columnas = 23


In [22]:
aeropuertos.schema

StructType(List(StructField(sigla,StringType,true),StructField(iata,StringType,true),StructField(nombre,StringType,true),StructField(municipio,StringType,true),StructField(departamento,StringType,true),StructField(categoria,StringType,true),StructField(latitud,IntegerType,true),StructField(longitud,StringType,true),StructField(propietario,StringType,true),StructField(explotador,StringType,true),StructField(longitud_pista,IntegerType,true),StructField(ancho_pista,DoubleType,true),StructField(pbmo,StringType,true),StructField(orientacion,StringType,true),StructField(elevacion,StringType,true),StructField(resolucion,StringType,true),StructField(fecha_construccion,StringType,true),StructField(fecha_vigencia,StringType,true),StructField(clase,StringType,true),StructField(tipo,StringType,true),StructField(numero_vuelos_origen,DoubleType,true),StructField(gcd_departamento,IntegerType,true),StructField(gcd_municipio,IntegerType,true)))

In [23]:
Counter([col[1] for col in aeropuertos.dtypes])

Counter({'string': 17, 'int': 4, 'double': 2})

En total tenemos 204.292 movimientos y tenemos en total 9 variables a revisar de las cuales 6 son nùmericas y las demás son cadenas de caracteres. Vale la pena aclarar que 7 variables son IDs.

### Tabla vuelosCopia

In [24]:
print(f"registros = {vuelos.count()}", f"columnas = {len(vuelos.columns)}")

registros = 467990 columnas = 13


In [25]:
vuelos.schema

StructType(List(StructField(ano,IntegerType,true),StructField(mes,StringType,true),StructField(origen,StringType,true),StructField(destino,StringType,true),StructField(tipo_equipo,StringType,true),StructField(tipo_vuelo,StringType,true),StructField(trafico,StringType,true),StructField(empresa,StringType,true),StructField(vuelos,IntegerType,true),StructField(sillas,IntegerType,true),StructField(carga_ofrecida,DoubleType,true),StructField(pasajeros,IntegerType,true),StructField(carga_bordo,DoubleType,true)))

In [26]:
Counter([col[1] for col in vuelos.dtypes])

Counter({'int': 4, 'string': 7, 'double': 2})

En total tenemos 204.292 movimientos y tenemos en total 9 variables a revisar de las cuales 6 son nùmericas y las demás son cadenas de caracteres. Vale la pena aclarar que 7 variables son IDs.

### Tabla proyeccionesCopia

In [27]:
print(f"registros = {proyecciones.count()}", f"columnas = {len(proyecciones.columns)}")

registros = 330 columnas = 5


In [28]:
proyecciones.schema

StructType(List(StructField(Anio,IntegerType,true),StructField(DP,IntegerType,true),StructField(DPNOM,StringType,true),StructField(Total Hombres,IntegerType,true),StructField(Total Mujeres,IntegerType,true)))

In [29]:
Counter([col[1] for col in proyecciones.dtypes])

Counter({'int': 4, 'string': 1})

En total tenemos 204.292 movimientos y tenemos en total 9 variables a revisar de las cuales 6 son nùmericas y las demás son cadenas de caracteres. Vale la pena aclarar que 7 variables son IDs.

### Tabla divipolaCopia

In [30]:
print(f"registros = {divipola.count()}", f"columnas = {len(divipola.columns)}")

registros = 9508 columnas = 12


In [31]:
divipola.schema

StructType(List(StructField(Código Departamento,IntegerType,true),StructField(Código Municipio,IntegerType,true),StructField(Código Centro Poblado,IntegerType,true),StructField(Nombre Departamento,StringType,true),StructField(Nombre Municipio,StringType,true),StructField(Nombre Centro Poblado,StringType,true),StructField(Tipo Centro Poblado,StringType,true),StructField(Longitud,DoubleType,true),StructField(Latitud,DoubleType,true),StructField(Nombre Distrito,StringType,true),StructField(Municipio/Áreas No Municipalizadas (ANM),StringType,true),StructField(Nombre Área Metropolitana,StringType,true)))

In [32]:
Counter([col[1] for col in divipola.dtypes])

Counter({'int': 3, 'string': 7, 'double': 2})

En total tenemos 204.292 movimientos y tenemos en total 9 variables a revisar de las cuales 6 son nùmericas y las demás son cadenas de caracteres. Vale la pena aclarar que 7 variables son IDs.

### Tabla pibCopia

In [33]:
print(f"registros = {pib.count()}", f"columnas = {len(pib.columns)}")

registros = 27 columnas = 10


In [34]:
pib.schema

StructType(List(StructField(CÃ³digo Departamento (DIVIPOLA),LongType,true),StructField(DEPARTAMENTOS,StringType,true),StructField(2006,LongType,true),StructField(2007,LongType,true),StructField(2008,DoubleType,true),StructField(2010,DoubleType,true),StructField(2011,LongType,true),StructField(2012,LongType,true),StructField(2013,DoubleType,true),StructField(2014,LongType,true)))

In [35]:
Counter([col[1] for col in pib.dtypes])

Counter({'bigint': 6, 'string': 1, 'double': 3})

En total tenemos 204.292 movimientos y tenemos en total 9 variables a revisar de las cuales 6 son nùmericas y las demás son cadenas de caracteres. Vale la pena aclarar que 7 variables son IDs.

## ¿Qué es una fila de cada fuente compartida?

### Tabla aeropuertosCopia

In [36]:
aeropuertos.where(aeropuertos['sigla'] == '7gv').show()

+-----+----+--------------------+--------------+------------+----------+-------+--------+--------------------+--------------------+--------------+-----------+------+-----------+---------+----------+------------------+--------------+-----+-------+--------------------+----------------+-------------+
|sigla|iata|              nombre|     municipio|departamento| categoria|latitud|longitud|         propietario|          explotador|longitud_pista|ancho_pista|  pbmo|orientacion|elevacion|resolucion|fecha_construccion|fecha_vigencia|clase|   tipo|numero_vuelos_origen|gcd_departamento|gcd_municipio|
+-----+----+--------------------+--------------+------------+----------+-------+--------+--------------------+--------------------+--------------+-----------+------+-----------+---------+----------+------------------+--------------+-----+-------+--------------------+----------------+-------------+
|  7gv|    |represa de las ca...|Paz de Ariporo|    casanare|AerÃ³dromo|    500|-71.2503|OLGA BRAVO DE 

En este caso un registro en específico representa un movimiento (transacción) que ha realizado un cliente con un producto específico. Cada movimiento cunta con identificadores únicos que detallan el producto, el tipo de transacción, el cliente, la factura, el proveedor y la orden de compra. Tambien indica la fecha en la que se ejecutó y la cantidad de productos del mismo tipo que se compraron para ese movimiento.

### Tabla vuelosCopia

In [37]:
vuelos.show(1)

+----+---+------+-------+-----------+----------+-------+--------+------+------+--------------+---------+-----------+
| ano|mes|origen|destino|tipo_equipo|tipo_vuelo|trafico| empresa|vuelos|sillas|carga_ofrecida|pasajeros|carga_bordo|
+----+---+------+-------+-----------+----------+-------+--------+------+------+--------------+---------+-----------+
|2004|  1|   bog|    mia|       B752|         R|      I|AMERICAN|     0|     0|           0.0|        0|        0.0|
+----+---+------+-------+-----------+----------+-------+--------+------+------+--------------+---------+-----------+
only showing top 1 row



En este caso un registro en específico representa un movimiento (transacción) que ha realizado un cliente con un producto específico. Cada movimiento cunta con identificadores únicos que detallan el producto, el tipo de transacción, el cliente, la factura, el proveedor y la orden de compra. Tambien indica la fecha en la que se ejecutó y la cantidad de productos del mismo tipo que se compraron para ese movimiento.

### Tabla proyeccionesCopia

In [38]:
proyecciones.show(1)

+----+---+---------+-------------+-------------+
|Anio| DP|    DPNOM|Total Hombres|Total Mujeres|
+----+---+---------+-------------+-------------+
|2005| 25|antioquia|      2630787|      2830056|
+----+---+---------+-------------+-------------+
only showing top 1 row



En este caso un registro en específico representa un movimiento (transacción) que ha realizado un cliente con un producto específico. Cada movimiento cunta con identificadores únicos que detallan el producto, el tipo de transacción, el cliente, la factura, el proveedor y la orden de compra. Tambien indica la fecha en la que se ejecutó y la cantidad de productos del mismo tipo que se compraron para ese movimiento.

### Tabla divipolaCopia

In [39]:
divipola.show(1)

+-------------------+----------------+---------------------+-------------------+----------------+---------------------+-------------------+--------------+--------------+---------------+----------------------------------------+-------------------------+
|Código Departamento|Código Municipio|Código Centro Poblado|Nombre Departamento|Nombre Municipio|Nombre Centro Poblado|Tipo Centro Poblado|      Longitud|       Latitud|Nombre Distrito|Municipio/Áreas No Municipalizadas (ANM)|Nombre Área Metropolitana|
+-------------------+----------------+---------------------+-------------------+----------------+---------------------+-------------------+--------------+--------------+---------------+----------------------------------------+-------------------------+
|                 41|           91001|             91001033|           amazonas|           NEIVA| COMUNIDAD INDÍGEN...|     CENTRO POBLADO|-70.0285785394|-4.11198257304|               |                               MUNICIPIO|               

En este caso un registro en específico representa un movimiento (transacción) que ha realizado un cliente con un producto específico. Cada movimiento cunta con identificadores únicos que detallan el producto, el tipo de transacción, el cliente, la factura, el proveedor y la orden de compra. Tambien indica la fecha en la que se ejecutó y la cantidad de productos del mismo tipo que se compraron para ese movimiento.

### Tabla pibCopia

In [40]:
pib.show(1)

+-------------------------------+-------------+-------+-------+-----------------+----------------+--------+--------+--------------------+--------+
|CÃ³digo Departamento (DIVIPOLA)|DEPARTAMENTOS|   2006|   2007|             2008|            2010|    2011|    2012|                2013|    2014|
+-------------------------------+-------------+-------+-------+-----------------+----------------+--------+--------+--------------------+--------+
|                             41|         uila|6895658|7441987|8686536.103258422|9548624.85406653|11113741|11400169|1.1967750469189592E7|12755711|
+-------------------------------+-------------+-------+-------+-----------------+----------------+--------+--------+--------------------+--------+
only showing top 1 row



En este caso un registro en específico representa un movimiento (transacción) que ha realizado un cliente con un producto específico. Cada movimiento cunta con identificadores únicos que detallan el producto, el tipo de transacción, el cliente, la factura, el proveedor y la orden de compra. Tambien indica la fecha en la que se ejecutó y la cantidad de productos del mismo tipo que se compraron para ese movimiento.

## Análisis descriptivo

### Tabla aeropuertosCopia

In [41]:
aeropuertos.summary().show()

+-------+-----+----+--------+---------+------------+----------+-------+------------------+--------------------+-------------+--------------+------------------+------------------+-----------+------------------+----------+------------------+--------------+------------------+--------+--------------------+------------------+-----------------+
|summary|sigla|iata|  nombre|municipio|departamento| categoria|latitud|          longitud|         propietario|   explotador|longitud_pista|       ancho_pista|              pbmo|orientacion|         elevacion|resolucion|fecha_construccion|fecha_vigencia|             clase|    tipo|numero_vuelos_origen|  gcd_departamento|    gcd_municipio|
+-------+-----+----+--------+---------+------------+----------+-------+------------------+--------------------+-------------+--------------+------------------+------------------+-----------+------------------+----------+------------------+--------------+------------------+--------+--------------------+---------------

El calculo de la media para las columnas ID, esto se hace automatico pues son columnas cuyo tipo de dato es numerico, sin embargo estas no son variables numericas por lo cual calcular y/o analizar la media no tiene sentido. La lógica es similar para los otros estadísticos.

En las estadíticas de Cantidad podemos observar que el valor promedio es 719.500, pero su desviación estandar es muy elevada (4729.367) por lo que se puede notar una gran dispersion en los datos.


reporte = ProfileReport(aeropuertos.toPandas())
reporte.to_file(PATH+"reporte_aeropuertos.html")

Vemos que la inmesa mayoria de movimientos se hicieron en 2015.

### Tabla vuelosCopia

In [42]:
vuelos.summary().show()

+-------+------------------+------------------+------+-------+------------------+----------+-------+------------------+------------------+-----------------+-----------------+------------------+------------------+
|summary|               ano|               mes|origen|destino|       tipo_equipo|tipo_vuelo|trafico|           empresa|            vuelos|           sillas|   carga_ofrecida|         pasajeros|       carga_bordo|
+-------+------------------+------------------+------+-------+------------------+----------+-------+------------------+------------------+-----------------+-----------------+------------------+------------------+
|  count|            467990|            467990|115441| 115441|            467990|    467990| 467990|            467990|            467990|           467990|           467987|            467990|            467971|
|   mean|2009.0125451398535|6.5515397665929775|  null|   null| 627.5512585591667|      null|   null|              null|10.101587640761554|723.759932

El calculo de la media para las columnas ID, esto se hace automatico pues son columnas cuyo tipo de dato es numerico, sin embargo estas no son variables numericas por lo cual calcular y/o analizar la media no tiene sentido. La lógica es similar para los otros estadísticos.

En las estadíticas de Cantidad podemos observar que el valor promedio es 719.500, pero su desviación estandar es muy elevada (4729.367) por lo que se puede notar una gran dispersion en los datos.


reporte = ProfileReport(vuelos.toPandas())
reporte.to_file(PATH+"reporte_vuelos.html")

Vemos que la inmesa mayoria de movimientos se hicieron en 2015.

### Tabla proyeccionesCopia

In [43]:
proyecciones.summary().show()

+-------+-----------------+----+--------+-----------------+------------------+
|summary|             Anio|  DP|   DPNOM|    Total Hombres|     Total Mujeres|
+-------+-----------------+----+--------+-----------------+------------------+
|  count|              330| 330|     330|              330|               330|
|   mean|           2009.5|25.0|    null|650666.4242424242| 843740.4454545454|
| stddev|2.876643180486918| 0.0|    null|751024.7851013226|3097897.0672473223|
|    min|             2005|  25|amazonas|            13825|             11636|
|    25%|             2007|  25|    null|           173396|            170374|
|    50%|             2009|  25|    null|           477977|            489035|
|    75%|             2012|  25|    null|           777720|            795592|
|    max|             2014|  25| vichada|          3476538|          55000000|
+-------+-----------------+----+--------+-----------------+------------------+



El calculo de la media para las columnas ID, esto se hace automatico pues son columnas cuyo tipo de dato es numerico, sin embargo estas no son variables numericas por lo cual calcular y/o analizar la media no tiene sentido. La lógica es similar para los otros estadísticos.

En las estadíticas de Cantidad podemos observar que el valor promedio es 719.500, pero su desviación estandar es muy elevada (4729.367) por lo que se puede notar una gran dispersion en los datos.


reporte = ProfileReport(proyecciones.toPandas())
reporte.to_file(PATH+"reporte_proyecciones.html")

Vemos que la inmesa mayoria de movimientos se hicieron en 2015.

### Tabla divipolaCopia

In [44]:
divipola.summary().show()

+-------+-------------------+------------------+---------------------+-------------------+----------------+---------------------+-------------------+------------------+------------------+--------------------+----------------------------------------+-------------------------+
|summary|Código Departamento|  Código Municipio|Código Centro Poblado|Nombre Departamento|Nombre Municipio|Nombre Centro Poblado|Tipo Centro Poblado|          Longitud|           Latitud|     Nombre Distrito|Municipio/Áreas No Municipalizadas (ANM)|Nombre Área Metropolitana|
+-------+-------------------+------------------+---------------------+-------------------+----------------+---------------------+-------------------+------------------+------------------+--------------------+----------------------------------------+-------------------------+
|  count|               9508|              9508|                 9508|               9508|            9508|                 9508|               9508|              9508|    

El calculo de la media para las columnas ID, esto se hace automatico pues son columnas cuyo tipo de dato es numerico, sin embargo estas no son variables numericas por lo cual calcular y/o analizar la media no tiene sentido. La lógica es similar para los otros estadísticos.

En las estadíticas de Cantidad podemos observar que el valor promedio es 719.500, pero su desviación estandar es muy elevada (4729.367) por lo que se puede notar una gran dispersion en los datos.


reporte = ProfileReport(divipola.toPandas())
reporte.to_file(PATH+"reporte_divipola.html")

Vemos que la inmesa mayoria de movimientos se hicieron en 2015.

### Tabla pibCopia

In [45]:
pib.summary().show()

+-------+-------------------------------+-------------+------------------+-----------------+-------------------+--------------------+-----------------+--------------------+--------------------+--------------------+
|summary|CÃ³digo Departamento (DIVIPOLA)|DEPARTAMENTOS|              2006|             2007|               2008|                2010|             2011|                2012|                2013|                2014|
+-------+-------------------------------+-------------+------------------+-----------------+-------------------+--------------------+-----------------+--------------------+--------------------+--------------------+
|  count|                             27|           27|                27|               27|                 27|                  27|               27|                  27|                  27|                  27|
|   mean|               48.7037037037037|         null| 6413240.740740741|7130828.037037037|  7851694.186446355|   8737185.290427044|9728735

El calculo de la media para las columnas ID, esto se hace automatico pues son columnas cuyo tipo de dato es numerico, sin embargo estas no son variables numericas por lo cual calcular y/o analizar la media no tiene sentido. La lógica es similar para los otros estadísticos.

En las estadíticas de Cantidad podemos observar que el valor promedio es 719.500, pero su desviación estandar es muy elevada (4729.367) por lo que se puede notar una gran dispersion en los datos.


reporte = ProfileReport(pib.toPandas())
reporte.to_file(PATH+"reporte_pib.html")

Vemos que la inmesa mayoria de movimientos se hicieron en 2015.

# Análisis de calidad de datos

## Completitud

In [46]:
def contar_vacios(df):
    resultados = []
    for c in df.columns:
        vacios = df.filter(df[c].isNull()).count()
        if vacios!=0:
            print('número de vacíos para columna '+c+': '+str( vacios ))
            resultados.append(vacios)
    return resultados


In [47]:
def contar_blancos(df):
    resultados = []
    for c in df.columns:
        vacios = df.where(df[c] == '').count()
        if vacios!=0:
            print('número de vacíos para columna '+c+': '+str( vacios ))
            resultados.append(vacios)
    return resultados


In [48]:
def cardinalidad(df):
    resultado = {}
    for c in df.columns:
        cardinalidad = df.select(col(c)).distinct().count()
        if cardinalidad>=df.count()*0.5:
            resultado[c] = cardinalidad
    return resultado


### Tabla aeropuertosCopia

In [49]:
contar_vacios(aeropuertos)

[]

Podemos ver que no hay datos vacíos en la tabla movimientosCopia.

In [50]:
contar_blancos(aeropuertos)

número de vacíos para columna iata: 482
número de vacíos para columna categoria: 41
número de vacíos para columna longitud: 140
número de vacíos para columna propietario: 141
número de vacíos para columna explotador: 141
número de vacíos para columna pbmo: 194
número de vacíos para columna orientacion: 547
número de vacíos para columna elevacion: 141
número de vacíos para columna resolucion: 147
número de vacíos para columna fecha_construccion: 383


número de vacíos para columna fecha_vigencia: 460
número de vacíos para columna clase: 141
número de vacíos para columna tipo: 141


[482, 41, 140, 141, 141, 194, 547, 141, 147, 383, 460, 141, 141]

Pero igualmente podemos notar que hay 197.182 registros cuyo ProveedorID y OrdenDeCompraID es un espacio en blanco. Lo que se puede considerar un claro problema a la completitud de los datos.

In [51]:
cardinalidad(aeropuertos)

{'sigla': 443, 'nombre': 413, 'longitud': 324}

Respecto a cardinalidad, solo la columna TransaccionProductoID tiene una cardinalidad superior al 50%.

### Tabla vuelosCopia

In [52]:
contar_vacios(vuelos)

número de vacíos para columna origen: 352549


número de vacíos para columna destino: 352549


número de vacíos para columna carga_ofrecida: 3


número de vacíos para columna carga_bordo: 19


[352549, 352549, 3, 19]

Podemos ver que no hay datos vacíos en la tabla movimientosCopia.

In [53]:
contar_blancos(vuelos)

[]

Pero igualmente podemos notar que hay 197.182 registros cuyo ProveedorID y OrdenDeCompraID es un espacio en blanco. Lo que se puede considerar un claro problema a la completitud de los datos.

In [54]:
cardinalidad(vuelos)

{}

### Tabla proyeccionesCopia

In [55]:
contar_vacios(proyecciones)

[]

Podemos ver que no hay datos vacíos en la tabla movimientosCopia.

In [56]:
contar_blancos(proyecciones)

[]

Pero igualmente podemos notar que hay 197.182 registros cuyo ProveedorID y OrdenDeCompraID es un espacio en blanco. Lo que se puede considerar un claro problema a la completitud de los datos.

In [57]:
cardinalidad(proyecciones)

{'Total Hombres': 329, 'Total Mujeres': 329}

### Tabla divipolaCopia

In [58]:
contar_vacios(divipola)

[]

Podemos ver que no hay datos vacíos en la tabla movimientosCopia.

In [59]:
contar_blancos(divipola)

número de vacíos para columna Nombre Distrito: 9134
número de vacíos para columna Nombre Área Metropolitana: 9028


[9134, 9028]

Pero igualmente podemos notar que hay 197.182 registros cuyo ProveedorID y OrdenDeCompraID es un espacio en blanco. Lo que se puede considerar un claro problema a la completitud de los datos.

In [60]:
cardinalidad(divipola)

{'Código Centro Poblado': 7879,
 'Nombre Centro Poblado': 5512,
 'Longitud': 7078,
 'Latitud': 7079}

### Tabla pibCopia

In [61]:
contar_vacios(pib)

[]

Podemos ver que no hay datos vacíos en la tabla movimientosCopia.

In [62]:
contar_blancos(pib)

[]

Pero igualmente podemos notar que hay 197.182 registros cuyo ProveedorID y OrdenDeCompraID es un espacio en blanco. Lo que se puede considerar un claro problema a la completitud de los datos.

In [63]:
cardinalidad(pib)

{'CÃ³digo Departamento (DIVIPOLA)': 17,
 'DEPARTAMENTOS': 17,
 '2006': 17,
 '2007': 17,
 '2008': 17,
 '2010': 17,
 '2011': 17,
 '2012': 17,
 '2013': 17,
 '2014': 17}

## Unicidad y Consistencia

In [64]:
def contar_registros_repetidos(movimientos):
    print(f"registros = {movimientos.count()}", f"registros distintos = {movimientos.distinct().count()}", f"registros repetidos = {movimientos.count() - movimientos.distinct().count()}")
    return None

In [65]:
def obtener_ids(movimientos):
    registros_unicos = movimientos.distinct().count()
    for c in movimientos.columns:
        registros_unicos_col =movimientos.select(col(c)).distinct().count()
        if registros_unicos - registros_unicos_col ==0:
            print(f'La columna {c} puede ser un ID')
    return None

### Tabla aeropuertosCopia

In [66]:
contar_registros_repetidos(aeropuertos)

registros = 547 registros distintos = 444 registros repetidos = 103


Como pueden ver hay 173.659 registros de movimientoos que son unicos, lo que implica que 30.633 movimientos son completamente repetidas.

In [67]:
obtener_ids(aeropuertos)

### Tabla vuelosCopia

In [68]:
contar_registros_repetidos(vuelos)

registros = 467990 registros distintos = 416312 registros repetidos = 51678


Como pueden ver hay 173.659 registros de movimientoos que son unicos, lo que implica que 30.633 movimientos son completamente repetidas.

In [69]:
obtener_ids(vuelos)

### Tabla proyeccionesCopia

In [70]:
contar_registros_repetidos(proyecciones)

registros = 330 registros distintos = 330 registros repetidos = 0


Como pueden ver hay 173.659 registros de movimientoos que son unicos, lo que implica que 30.633 movimientos son completamente repetidas.

In [71]:
obtener_ids(proyecciones)

### Tabla divipolaCopia

In [72]:
contar_registros_repetidos(divipola)

registros = 9508 registros distintos = 9508 registros repetidos = 0


Como pueden ver hay 173.659 registros de movimientoos que son unicos, lo que implica que 30.633 movimientos son completamente repetidas.

In [73]:
obtener_ids(divipola)

### Tabla pibCopia

In [74]:
contar_registros_repetidos(pib)

registros = 27 registros distintos = 17 registros repetidos = 10


Como pueden ver hay 173.659 registros de movimientoos que son unicos, lo que implica que 30.633 movimientos son completamente repetidas.

In [75]:
obtener_ids(pib)

La columna CÃ³digo Departamento (DIVIPOLA) puede ser un ID
La columna DEPARTAMENTOS puede ser un ID
La columna 2006 puede ser un ID
La columna 2007 puede ser un ID
La columna 2008 puede ser un ID
La columna 2010 puede ser un ID
La columna 2011 puede ser un ID
La columna 2012 puede ser un ID
La columna 2013 puede ser un ID
La columna 2014 puede ser un ID


# Conclusiones/resultados

## ¿Es posible resolver los análisis basados en tableros de control propuestos?

In [76]:
data = movimientos.where((movimientos['ProveedorID'] != '') & (movimientos['OrdenDeCompraID'] != '') & (movimientos['Cantidad'] > 0))
data.show(5)

NameError: name 'movimientos' is not defined

In [ ]:
data.count()

In [ ]:
print(f"porcentajen datos aptos = {(data.count()/movimientos.count())*100}")

La cantidad de datos optimos para hacer el tablero es de 7.110, lo que implica que el analisis se realizaria sobre el 3.4% de los datos entregados. Siendo esta una muestra muy corta y poco representativa, concluyo que no es recomendable hacer el analisis basado en el tablero de control.

## ¿Que representa la fila promedio de los datos analizados?

En este caso un registro en específico representa un movimiento (transacción) que ha realizado un cliente con un producto específico. Cada movimiento cunta con identificadores únicos que detallan el producto, el tipo de transacción, el cliente, la factura, el proveedor y la orden de compra. Tambien indica la fecha en la que se ejecutó y la cantidad de productos del mismo tipo que se compraron para ese movimiento.